In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook

import sys 
sys.path.append('..')

from ParserQuery import *
from Index import Index
from Weighter import WeighterVector, WeighterSchema3
from IRModel import Vectoriel, LanguageModel, BM25Model
from Evaluation import *
from EvalIRModel import *

from diversity.Clustering import ClusteringDiversifier

%load_ext autoreload
%autoreload 2

In [ ]:
train_prop = 0.8
seed = 42
train_results = {}
test_results = {}

lambda_ = 0.13
k1 = 2
b = 0.95

In [ ]:
#'''
data_path = '../easyCLEF08/'
index_path = '../indexes/easyCLEF08/'
index_name = 'easyCLEF08'
#'''

'''
data_path = '../data/easyCLEF08/'
index_path = '../indexes/easyCLEF08/'
index_name = 'easyCLEF08'
'''

filename_queries = data_path+'/easyCLEF08_query.txt'
filename_jugements = data_path+'/easyCLEF08_gt.txt'

eval_ir = EvalIRModel(filename_queries, filename_jugements)

# Evaluate models
Compare baseline aux differents méthodes du diversite.

In [ ]:
index = Index('easyCLEF08', '../easyCLEF08/easyCLEF08_text.txt', index_path='../indexes/easyCLEF08/')
#index.indexation()
weighter2 = WeighterVector(index)
#weighter2.calculeNorms()
weighter3 = WeighterSchema3(index)
#weighter3.calculeNorms()
#baseline = LanguageModel(weighter2)
clust = ClusteringDiversifier(index)
#wrap_clust = lambda m, q: clust.diversify(m.getRanking(q, lambda_=lambda_))[0].values
baseline = BM25Model(weighter2)


In [ ]:
results = []
clusters_n = [1,3,5,10,15,20]
for nc in clusters_n:

    wrap_clust = lambda m, q: clust.diversify(m.getRanking(q, k1=k1, b=b), by_top_n=40,  n_clusters=nc)[0].values
    train_results = eval_ir.evalModel(baseline,
                                       ranking_call = wrap_clust,
                                       train_prop=train_prop, seed=seed, mode='train')
    results.append((train_results['precision_at_20'],train_results['cluster_recall_at_20']))

In [ ]:
baseline_result = eval_ir.evalModel(baseline,
                                       ranking_call =lambda m,q : m.getRanking(q, k1=k1, b=b),
                                       train_prop=train_prop, seed=seed, mode='train')

In [ ]:
baseline_result['cluster_recall_at_20']

In [ ]:
plt.figure()
results = np.array(results)
plt.plot(clusters_n, results[:,1])
plt.title("Clustering recall @20 avec diversite par clustering (les 40 plus pertinents)")
plt.ylabel('CR@20')
plt.xlabel("Nombre des clusters")
plt.axhline(y=baseline_result['cluster_recall_at_20'], label="Baseline")
plt.legend()
plt.savefig('diversite_clustering_clust_recall')

In [ ]:
plt.figure()
plt.plot(clusters_n, results[:,0])
plt.title("Précision @20 avec diversite par clustering (les 40 plus pertinents)")
plt.ylabel('Prec@20')
plt.xlabel("Nombre des clusters")
plt.axhline(y=baseline_result['precision_at_20'], label="Baseline")
plt.legend()
plt.savefig('diversite_clustering_precision')

In [ ]:
fig = plt.figure()
legend = []

ax1 = fig.add_subplot(2, 2, 1)
plt.title('En train')
i = 0
for model_name, model_results in train_results.items():
    legend.append(model_name)
    i += 1
    ax1.bar(i, model_results['precision_at_20'], yerr=model_results['precision_at_20_std'], capsize=4)
fig.legend(legend)
plt.show()

In [ ]:
del train_results['diversite_clust']

In [ ]:
qp = QueryParser()
qp.initFile('../easyCLEF08/easyCLEF08_query.txt', '../easyCLEF08/easyCLEF08_gt.txt')
qu = qp.nextQuery()
print(qu.text_, '\n', qu.relevants_)
prec = PrecisionAtN()
prec_cluster = ClusterRecallAtN()
rank = baseline.getRanking(qu.text_)

irlist = IRList(qu, rank)
print(prec.eval(ir_list=irlist))
print(prec_cluster.eval(irlist))

In [ ]:
cd = ClusteringDiversifier(index, cluster_order_by='size_descending')

data, cluster_order = cd.diversify(rank, by_top_n=30, n_clusters=6)
print(cluster_order)
data.values

In [ ]:
data[:,0]